## Data Description

Since my model is designed to regress the EOS between canola and corn, it requires data on the price and quantity of both these crops.

 
This is a time series dataset, merged from two sources.

**Raw Data:** 

- "Estimated areas, yield, production, average farm price and total farm value of principal field crops, in metric and imperial units": 

    - Provides production and yield data for Canadian crops.
    - Source: https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=3210035901

- "Raw materials price index for crop products, animals and animal products, monthly": Provides price indices for crop products on a monthly basis.

    - The price indices for crop products on a monthly basis. But is converted to yearly values ad hoc, by taking arithmetic average of monthly values.
    - Source: https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=1810026803&cubeTimeFrame.startMonth=01&cubeTimeFrame.startYear=1981&cubeTimeFrame.endMonth=09&cubeTimeFrame.endYear=2024&referencePeriods=19810101%2C20240901
    
    
<br />

- **Cleaned data: (at the end of this notebook)**

    - **year**: the year in which the observation is measured.

    - **canola_PI**: The price index for canola, with a base value of 100 in year 2020. This variable reflects the relative changes in the price of canola over time.

    - **corn_PI**: Similarly, the price index for corn, with a base of 100 in 2020.

    - **canola_Q**: The quantity of canola produced, measured in metric tonnes. This variable represents the annual production quantity of canola in Canada.	

    - **corn_Q**: The quantity of canola produced, in metric tonnes, represents the annual production quantity of canola in Canada.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

## Import datasets

In [2]:
data_Q = pd.read_excel('crops_production.xlsx', sheet_name='Sheet1',header=0)
data_PI =  pd.read_excel('Price_index_crops.xlsx', sheet_name='Sheet4')
data_Q.head()

,Unnamed: 0,1981,1982,1983,1984,1985,1986,1987,1988,1989,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Barley 10 11,13724150,13965000,10209000,10278600,12387000,14568000,13916100,10326100,11784100,...,8256600,8839400,7891300,8379700,10382600,10740600,6983941,9986681,8905385,7600182
1,Canary seed 10,..,..,..,..,..,126100,99600,59900,115500,...,148600,140300,145000,157600,175000,224700,127238,159306,112210,161843
2,Canola (rapeseed) 10 12,1848500,2218100,2593300,3411900,3497900,3713700,3719500,4218300,3209200,...,18376500,19599200,21458100,20723500,19912300,19484700,14248281,18849801,19191655,18980484
3,Chick peas 10,..,..,..,..,..,..,..,..,..,...,83500,75200,95600,311300,251500,214400,91288,145900,159100,327200
4,Corn for grain 10 13 14,6682600,6522300,5930800,6777500,6969900,5911700,7064700,5449500,6570900,...,13679500,13889000,14095500,13884800,13403900,13563300,14610792,14538878,15420909,15167588


In [4]:
data_PI.head()

,year,canola_PI,corn_PI
0,1981,60.233333,70.691667
1,1982,59.333333,58.991667
2,1983,66.025000,75.025000
3,1984,83.008333,84.175000
4,1985,68.425000,74.583333


### Transpose the quantity data

In [5]:
# Transpose the dataframe
data_Q_transposed = data_Q.T

# Set the first row as the header
data_Q_transposed.columns = data_Q_transposed.iloc[0]
data_Q_transposed = data_Q_transposed.drop(data_Q_transposed.index[0])

# Add a 'year' column 
data_Q_transposed.index.name = 'year'
# # set it as the index
data_Q_transposed.reset_index(inplace=True)
data_Q_transposed.head()

Unnamed: 0,year,Barley 10 11,Canary seed 10,Canola (rapeseed) 10 12,Chick peas 10,Corn for grain 10 13 14,Flaxseed 10 15,Lentils 10,Mustard seed 10 16,Oats 10 11,Soybeans 10 17,Sunflower seed 10 18,"Wheat, all 10 19 20","Wheat, durum 10","Wheat, spring 10","Wheat, winter remaining 10 21"
0,1981,13724150,..,1848500,..,6682600,467400,55700,98000,3188300,606800,165200,24802200,..,20750200,1075000
1,1982,13965000,..,2218100,..,6522300,751900,90200,76500,3636800,848000,94600,26714750,..,22939750,654000
2,1983,10209000,..,2593300,..,5930800,443900,57400,86400,2773300,735000,46200,26464200,..,22631100,1213100
3,1984,10278600,..,3411900,..,6777500,693500,38900,112400,2576100,917000,84800,21187900,..,17786600,1291300
4,1985,12387000,..,3497900,..,6969900,896900,62400,125300,2735700,1012000,61700,24252200,..,20519300,1772900


### Extract columns  for corn and canola quantities

In [6]:
# data_Q.transpose().index[1:]
data_Q2 = pd.DataFrame(columns=['year','canola_Q','corn_Q'])

data_Q2['year'] = data_Q_transposed.iloc[:,0]
data_Q2['canola_Q'] = data_Q_transposed.iloc[:,3]
data_Q2['corn_Q'] = data_Q_transposed.iloc[:,5]
# data_Q2.rename_axis("year") 
data_Q2.head()

,year,canola_Q,corn_Q
0,1981,1848500,6682600
1,1982,2218100,6522300
2,1983,2593300,5930800
3,1984,3411900,6777500
4,1985,3497900,6969900


## Merged Dataset

In [7]:
dt_merge = pd.merge(left=data_PI, 
                   right=data_Q2, 
                   how='inner', 
                   left_on='year', 
                   right_on='year')
dt_merge = dt_merge.dropna()
dt_merge.head()

,year,canola_PI,corn_PI,canola_Q,corn_Q
0,1981,60.233333,70.691667,1848500,6682600
1,1982,59.333333,58.991667,2218100,6522300
2,1983,66.025000,75.025000,2593300,5930800
3,1984,83.008333,84.175000,3411900,6777500
4,1985,68.425000,74.583333,3497900,6969900


In [8]:
dt_merge.to_csv("corn_canola_pq.csv")